In [95]:
def ObjectView(Obj,**Kwargs):
    TypeMap={tuple:"tuple",list:"list",dict:"dict",set:"set",frozenset:"frozenset"}
    FlagNoSplit = "noSplit" in Kwargs and Kwargs["noSplit"] == True
    ThemeMap = dict((
    ("tuple",{"startB0":"=","startB1":"=","start0":"=","start1":"=","startB>":"╔","start>":"╦",
    "continue":"║","add":"╠","end":"╚"}),
    ("list",{"startB0":"~","startB1":"~","start0":"~","start1":"~","startB>":"╓","start>":"╥",
    "continue":"║","add":"╟","end":"╙"}),
    ("dict",{"startB0":"╫","startB1":"╫","start0":"╫","start1":"╫","startB>":"╓","start>":"╥","startI":"╥",
    "continue":"║","add":"╟","end":"╙","endI":"╰"}),
    ("set",{"startB0":"┼","startB1":"┼","start0":"┼","start1":"┼","startB>":"┌","start>":"┬",
    "continue":"│","add":"├","end":"└"}),
    ("frozenset",{"startB0":"╪","startB1":"╪","start0":"╪","start1":"╪","startB>":"╒","start>":"╤",
    "continue":"│","add":"╞","end":"╘"}),
    ("str",{"start":'"',"end":"","spacingL":'"',"spacingR":""}),("spacing"," ")))
    # although python dict support objects as keys, strings are used for interoperability
    if "theme" in Kwargs and Kwargs["theme"] != None:
        for _k,_p in Kwargs["theme"].items():
            if type(_k) != str: continue
            if _k in {"tuple","list","dict","set","frozenset","str"}:
                for __k,__p in _p.items():
                    ThemeMap[_k][__k] = __p
            elif _k in {"spacing",}:
                ThemeMap[_k] = _p
    for _v in ("tuple","list","dict","set","frozenset"):# will not nest until the default theme is fixed
        ThemeMap[ThemeMap[_v]["startB>"]]=ThemeMap[_v]["continue"]
        ThemeMap[ThemeMap[_v]["start>"]]=ThemeMap[_v]["continue"]
        ThemeMap[ThemeMap[_v]["continue"]]=ThemeMap[_v]["continue"]
        ThemeMap[ThemeMap[_v]["add"]]=ThemeMap[_v]["continue"]
        if _v == "dict": ThemeMap[ThemeMap[_v]["startI"]]=ThemeMap[_v]["continue"]
    AdjoinSet = set()
    for _v in ("tuple","list","dict","set","frozenset"):
        AdjoinSet.add(ThemeMap[_v]["startB>"])
        AdjoinSet.add(ThemeMap[_v]["start>"])
        AdjoinSet.add(ThemeMap[_v]["continue"])
        AdjoinSet.add(ThemeMap[_v]["add"])
        if _v == "dict": AdjoinSet.add(ThemeMap[_v]["startI"])
    Res = [""]
    Types = [None,]
    Lens = [1,]
    Leaves = []
    Queue = [iter((Obj,))]
    while True:
        Temp = next(Queue[-1])
        Lens[-1] -= 1
        TypeTemp = type(Temp)
        FlagIter = TypeTemp in {tuple,list,dict,set,frozenset}
        if FlagIter and len(Temp) != 0:
            Types.append(TypeTemp)
            Lens.append(len(Temp) * (2 if TypeTemp == dict else 1))
            if TypeTemp == dict:
                Res[-1] += ThemeMap[TypeMap[TypeTemp]]["start"+("B" if len(Queue) == 1 else "")\
                +("1" if Lens[-1] == 2 else ">")]+ThemeMap[TypeMap[TypeTemp]]["startI"]
            else:
                Res[-1] += ThemeMap[TypeMap[TypeTemp]]["start"+("B" if len(Queue) == 1 else "")+("1" if Lens[-1] == 1 else ">")]
            Queue.append(iter((_c for _r in Temp.items() for _c in _r) if TypeTemp == dict else Temp))
        else:
            NewRows = len(Leaves)
            if FlagNoSplit:
                Leaves.append((ThemeMap[TypeMap[TypeTemp]]["start"+("B" if len(Queue) == 1 else "")+"0"] if FlagIter\
                                else (ThemeMap["str"]["start"]+Temp+ThemeMap["str"]["end"]) if TypeTemp == str\
                                else str(Temp)).replace("\n","\\n"))
            else:
                Leaves.extend((ThemeMap[TypeMap[TypeTemp]]["start"+("B" if len(Queue) == 1 else "")+"0"] if FlagIter\
                                else (ThemeMap["str"]["start"]+Temp+ThemeMap["str"]["end"])\
                               .replace("\n",ThemeMap["str"]["spacingR"]+"\n"+ThemeMap["str"]["spacingL"]) if TypeTemp == str\
                                else str(Temp)).splitlines())
            NewRows = len(Leaves) - NewRows
            AddRow = "".join(ThemeMap.get(_v,ThemeMap["spacing"]) for _v in Res[-1])
            Res.extend((AddRow,)*(NewRows-1))
            while Lens[-1] == 0:
                Lens.pop()
                Types.pop()
                Queue.pop()
                if len(Queue) <= 1: return "\n".join(Branch+Leaf for Branch,Leaf in zip(Res,Leaves)) 
            FindTemp = len(Res[-1]) - 1
            while Res[-1][FindTemp] not in AdjoinSet:
                # FindTemp will never reach below 0 as there is always a root
                FindTemp -= 1
            if Types[-1] == dict:
                Res.append(AddRow[:FindTemp]\
                            +(ThemeMap[TypeMap[Types[-1]]]["endI"] if Lens[-1] % 2 == 1\
                            else (ThemeMap[TypeMap[Types[-1]]]["end" if Lens[-1] == 2 else "add"]\
                            +ThemeMap[TypeMap[Types[-1]]]["startI"])))
            elif Types[-1] in {tuple,list,set,frozenset}:
                Res.append(AddRow[:FindTemp]+ThemeMap[TypeMap[Types[-1]]]["end" if Lens[-1] == 1 else "add"])

In [99]:
print("Result:\n"+ObjectView({}))
print("Result:\n"+ObjectView("A\njob\nwell\ndone"))
Val = {(((),),):{-0.0:[[[-0.0,]],[],set()]},frozenset((frozenset((frozenset(),)),)):set((frozenset(),)),None:{"Not\nbad\nat\nall":([("A\njob\nwell\ndone",)],),False:abs,True:dict()}}
print("Result:\n"+ObjectView(Val))
Val = frozenset((1,2,3,(),frozenset(((("a","b","c"),("000","111","222")),))))
print("Result:\n"+ObjectView((Val,(((ObjectView(Val),),),(Val,)))))
Theme = {"tuple":{"startB0":"()","start0":"()","startB1":"═","start1":"═"},
         "list":{"startB0":"[]","start0":"[]","startB1":"─","start1":"─"},
         "dict":{"startB0":"{:}","start0":"{:}","startI":"╥","endI":"╙"},
         "set":{"startB0":"{}","start0":"{}"},
         "frozenset":{"startB0":"{#}","start0":"{#}"},
        "str":{"start":"'","end":"'","spacingL":"█","spacingR":"\\n"},"spacing":"█"}
print("Theme:\n"+ObjectView(Theme))
print("Result using a theme:\n"+ObjectView(Val2,theme=Theme))
print("Result using a theme without string splitting:\n"+ObjectView(Val2,theme=Theme,noSplit=True))

Result:
╫
Result:
"A
"job
"well
"done
Result:
╓╥===
║╰╫╥-0.0
║  ╰╥~~-0.0
║   ╟~
║   ╙┼
╟╥╪╪╪
║╰┼╪
╙╥None
 ╰╥╥"Not
  ║║"bad
  ║║"at
  ║║"all
  ║╰=~="A
  ║    "job
  ║    "well
  ║    "done
  ╟╥False
  ║╰<built-in function abs>
  ╙╥True
   ╰╫
Result:
╔╤1
║╞2
║╞3
║╞╪╦╦"a
║│ ║╠"b
║│ ║╚"c
║│ ╚╦"000
║│  ╠"111
║│  ╚"222
║╘=
╚╦=="╒1
 ║  "╞2
 ║  "╞3
 ║  "╞╪╦╦"a
 ║  "│ ║╠"b
 ║  "│ ║╚"c
 ║  "│ ╚╦"000
 ║  "│  ╠"111
 ║  "│  ╚"222
 ║  "╘=
 ╚=╤1
   ╞2
   ╞3
   ╞╪╦╦"a
   │ ║╠"b
   │ ║╚"c
   │ ╚╦"000
   │  ╠"111
   │  ╚"222
   ╘=
Theme:
╓╥"tuple
║╰╥╥"startB0
║ ║╰"()
║ ╟╥"start0
║ ║╰"()
║ ╟╥"startB1
║ ║╰"═
║ ╙╥"start1
║  ╰"═
╟╥"list
║╰╥╥"startB0
║ ║╰"[]
║ ╟╥"start0
║ ║╰"[]
║ ╟╥"startB1
║ ║╰"─
║ ╙╥"start1
║  ╰"─
╟╥"dict
║╰╥╥"startB0
║ ║╰"{:}
║ ╟╥"start0
║ ║╰"{:}
║ ╟╥"startI
║ ║╰"╥
║ ╙╥"endI
║  ╰"╙
╟╥"set
║╰╥╥"startB0
║ ║╰"{}
║ ╙╥"start0
║  ╰"{}
╟╥"frozenset
║╰╥╥"startB0
║ ║╰"{#}
║ ╙╥"start0
║  ╰"{#}
╟╥"str
║╰╥╥"start
║ ║╰"'
║ ╟╥"end
║ ║╰"'
║ ╟╥"spacingL
║ ║╰"█
║ ╙╥"spacingR
║  ╰"\n
╙╥"spacing
 ╰"█
Result